In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with CAMB.AI Models

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/cambai_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial/generative_ai/cambai_intro.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/cambai_intro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/cambai_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

### Camb AI on Vertex AI

You can deploy the Camb AI models in your own endpoint.

### Available Camb AI models

#### MARS7
(Multilingual AutoRegressive Speech 7) is the latest generation in CAMB.AI’s MARS series of speech synthesis models. MARS7 creates hyper-realistic, prosodic, multilingual text-to-speech (TTS) outputs, featuring optional voice cloning and fine-grained emotional control. Deployed on Google Cloud’s Vertex AI Launchpad, it brings near real-time latency, with an architectural focus on parameter efficiency and global context understanding.

## Objective
This notebook shows how to use **Vertex AI API** to deploy the Camb AI models.


## Get Started


### Install required packages


In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

In [2]:
! pip3 install -U -q httpx

In [ ]:
! apt-get install jq -y

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [4]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Select one of Camb AI models

In [2]:
PUBLISHER_NAME = "cambai"  # @param {type:"string"}
PUBLISHER_MODEL_NAME = "mars7"
available_regions = ["us-central1"]

### Select a location and a version from the dropdown

In [3]:
import ipywidgets as widgets
from IPython.display import display

dropdown_loc = widgets.Dropdown(
    options=available_regions,
    description="Select a location:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_loc_eventhandler(change):
    global LOCATION
    if change["type"] == "change" and change["name"] == "value":
        LOCATION = change.new
        print("Selected:", change.new)


LOCATION = dropdown_loc.value
dropdown_loc.observe(dropdown_loc_eventhandler, names="value")
display(dropdown_loc)

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [11]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")

### Import required libraries

In [5]:
import base64
import json
import time

## Using Vertex AI API

### Upload Model

In [ ]:
UPLOAD_MODEL_PAYLOAD = {
    "model": {
        "displayName": "ModelGarden_LaunchPad_Model_" + time.strftime("%Y%m%d-%H%M%S"),
        "baseModelSource": {
            "modelGardenSource": {
                "publicModelName": f"publishers/{PUBLISHER_NAME}/models/{PUBLISHER_MODEL_NAME}",
            }
        },
    }
}

request = json.dumps(UPLOAD_MODEL_PAYLOAD)

! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/models:upload -d '{request}'

### Get Model

In [ ]:
MODEL_ID = [your - model - id]  # @param {type: "number"}

! curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/models/{MODEL_ID}

### Create Endpoint

In [ ]:
CREATE_ENDPOINT_PAYLOAD = {
    "displayName": "ModelGarden_LaunchPad_Endpoint_" + time.strftime("%Y%m%d-%H%M%S"),
}

request = json.dumps(CREATE_ENDPOINT_PAYLOAD)

! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints -d '{request}'

### Get Endpoint

In [ ]:
ENDPOINT_ID = [your - endpoint - id]  # @param {type: "number"}

! curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}

### Deploy Model

In [19]:
MACHINE_TYPE = "g2-standard-4"  # @param {type: "string"}
ACCELERATOR_TYPE = "NVIDIA_L4"  # @param {type: "string"}
ACCELERATOR_COUNT = 1  # @param {type: "number"}

In [ ]:
DEPLOY_PAYLOAD = {
    "deployedModel": {
        "model": f"projects/{PROJECT_ID}/locations/{LOCATION}/models/{MODEL_ID}",
        "displayName": "ModelGarden_LaunchPad_DeployedModel_"
        + time.strftime("%Y%m%d-%H%M%S"),
        "dedicatedResources": {
            "machineSpec": {
                "machineType": MACHINE_TYPE,
                "acceleratorType": ACCELERATOR_TYPE,
                "acceleratorCount": ACCELERATOR_COUNT,
            },
            "minReplicaCount": 1,
            "maxReplicaCount": 1,
        },
    },
    "trafficSplit": {"0": 100},
}

request = json.dumps(DEPLOY_PAYLOAD)
print("Request payload to Deploy Model:")
print(json.dumps(DEPLOY_PAYLOAD, indent=2))
print("\nResult:")
! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:deployModel -d '{request}'

### Generate audio

#### Unary call

Sends a POST request to the specified API endpoint to get a response from the model for a joke using the provided payload.

In [ ]:
!curl -o ref.flac "https://storage.googleapis.com/cambai-prod-public/public/ref.flac"
print("Downloaded ref.flac")

text = "The quick brown fox jumps over the lazy dog"
audio_ref_path = "ref.flac"
ref_text = None
language = "en-us"

# Prepare the JSON payload in a separate file
json_payload_file = "payload.json"

with open(audio_ref_path, "rb") as audio_file:
    encoded_string = base64.b64encode(audio_file.read()).decode("utf-8")

payload = {
    "text": text,
    "audio_ref": encoded_string,
    "ref_text": ref_text,
    "language": language,
}

with open(json_payload_file, "w") as f:
    json.dump(payload, f, indent=2)

save_output_cmd = f"""
curl -X POST {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:rawPredict \\
        -H "Content-Type: application/json" \\
        -H "Authorization: Bearer $(gcloud auth print-access-token)" \\
        -d @{json_payload_file} > response.json

# save audio
cat response.json | jq -r '.predictions[0]' | base64 -d > output.flac
"""
print(save_output_cmd)
!{save_output_cmd}

In [ ]:
import soundfile as sf
import tempfile
from IPython.display import Audio

def display_flac_in_notebook(path_to_flac):
    data, samplerate = sf.read(path_to_flac)
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        sf.write(tmp.name, data, samplerate, format='WAV')
        return Audio(filename=tmp.name)

display_flac_in_notebook("output.flac")

## Using Vertex AI SDK for *Python*

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### Upload Model

In [ ]:
model = aiplatform.Model.upload(
    display_name="ModelGarden_LaunchPad_Endpoint_" + time.strftime("%Y%m%d-%H%M%S"),
    model_garden_source_model_name=f"publishers/{PUBLISHER_NAME}/models/{PUBLISHER_MODEL_NAME}",
)

### Create Endpoint

In [ ]:
my_endpoint = aiplatform.Endpoint.create(
    display_name="ModelGarden_LaunchPad_Endpoint_" + time.strftime("%Y%m%d-%H%M%S")
)

### Deploy Model

In [ ]:
MACHINE_TYPE = "g2-standard-4"  # @param {type: "string"}
ACCELERATOR_TYPE = "NVIDIA_L4"  # @param {type: "string"}
ACCELERATOR_COUNT = 1  # @param {type: "number"}

In [ ]:
model.deploy(
    endpoint=my_endpoint,
    deployed_model_display_name="ModelGarden_LaunchPad_DeployedModel_"
    + time.strftime("%Y%m%d-%H%M%S"),
    traffic_split={"0": 100},
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    min_replica_count=1,
    max_replica_count=1,
)

### Generate audio

#### Unary call

In [ ]:
def predict_custom_trained_model_sample(
    instances: dict, target_file: str = "output.flac"
):
    # If you created your endpoint using a different method you can set the value by uncommenting the next line and making sure ENDPOINT_ID is set
    # my_endpoint = aiplatform.Endpoint(f'{ENDPOINT_ID}')
    DATA = {"instances": [instances]}
    response = my_endpoint.raw_predict(
        body=json.dumps(DATA).encode("utf-8"),
        headers={"Content-Type": "application/json"},
    )
    with open(target_file, "wb") as f:
        audio_bytes = base64.b64decode(json.loads(response.content)["predictions"][0])
        f.write(audio_bytes)


with open("ref.flac", "rb") as f:
    audio_ref_bytes = base64.b64encode(f.read()).decode("utf-8")

predict_custom_trained_model_sample(
    instances={
        "text": "The quick brown fox jumps over the lazy dog",
        "audio_ref": audio_ref_bytes,
        "ref_text": None,
        "language": "en-us",
    },
    target_file="output.flac",
)

## Cleaning up

##### Cleaning up deployment resources
To clean up all Google Cloud resources used in this notebook, you can delete the Google Cloud project you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

* Model
* Endpoint


##### Undeploy model

In [ ]:
UNDEPLOY_PAYLOAD = {"deployedModelId": MODEL_ID}

request = json.dumps(UNDEPLOY_PAYLOAD)

! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:undeployModel -d '{request}'

##### Delete Endpoint

In [ ]:
! curl -X DELETE -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}